<a href="https://colab.research.google.com/github/ashispapu/LLMs/blob/main/MambaAudioFinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!mkdir checkpoints

In [ ]:
!git clone https://huggingface.co/fnlp/SpeechTokenizer
!pip3 install speechtokenizer



Cloning into 'SpeechTokenizer'...
remote: Enumerating objects: 17, done.
remote: Total 17 (delta 0), reused 0 (delta 0), pack-reused 17
Unpacking objects: 100% (17/17), 348.04 KiB | 4.58 MiB/s, done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 3.1 MB/s eta 0:00:00


In [ ]:
!pip install datasets
!pip install causal-conv1d==1.0.0
!pip install mamba-ssm==1.0.1
!pip install huggingface_hub
!pip install torchinfo
!pip install wandb

!export LC_ALL="en_US.UTF-8"
!export LD_LIBRARY_PATH="/usr/lib64-nvidia"
!export LIBRARY_PATH="/usr/local/cuda/lib64/stubs"
!ldconfig /usr/lib64-nvidia

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 3.9 MB/s eta 0:00:00
  Created wheel for causal-conv1d: filename=causal_conv1d-1.0.0-cp310-cp310-linux_x86_64.whl size=9116761 sha256=4bbd2c2672ecd02c1e43f8e52552de593099619abc6dda18b2ac650e08110124
  Stored in directory: /root/.cache/pip/wheels/9a/48/f5/eb0c6d6d8e00131eaa57927b537a23832b37e2f01b801d9c5d
Successfully built causal-conv1d
  Preparing metadata (setup.py) ... done
  Created wheel for mamba-ssm: filename=mamba_ssm-1.0.1-cp310-cp310-linux_x86_64.whl size=137567739 sha256=1775b610f76d6bc71ffaa72375df8c0afde52c1c14f1c788ad6afee4290adff2
  Stored in directory: /root/.cache/pip/wheels/08/cf/65/cc589985f9689241fe2c154ce1c60738f58a24e76ce

In [ ]:
!pip install pytube
!pip install moviepy
!pip install soundfile

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 1.9 MB/s eta 0:00:00


In [ ]:
!wandb login

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: ERROR API key must be 40 characters long, yours was 37
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
from speechtokenizer import SpeechTokenizer
import soundfile as sf
import torchaudio
import torch
import numpy as np
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

NUM_QUANTIZERS_USED = 4
batch_size = 4

config_path = '/content/SpeechTokenizer/speechtokenizer_hubert_avg/config.json'
ckpt_path = '/content/SpeechTokenizer/speechtokenizer_hubert_avg/SpeechTokenizer.pt'
speech_tokenizer = SpeechTokenizer.load_from_checkpoint(config_path, ckpt_path).to(device)
speech_tokenizer.eval()


def normalize_waveform(waveform, sr):
    if len(waveform.shape) == 2 and waveform.shape[1] > 0:
        waveform = torch.mean(waveform, dim=0, keepdim=True)
    waveform = waveform.reshape(1, -1)
    waveform = torchaudio.functional.resample(waveform, sr, speech_tokenizer.sample_rate)
    return waveform


# Expected input: waveform of shape (B, C, V)
# The encoding returns shape (num codebooks, 1, timestep). Using it with batch size > 1 gave me a lot of errors so I didn't try it.
# After encoding the tokens, I threw away some of the later codebooks which encode less information before I flattened them as explained below.
def tokenize(waveform):
    with torch.no_grad():
        codes = speech_tokenizer.encode(waveform.unsqueeze(0).to(device))  # codes: (n_q, B, T)
    semantic_tokens = codes[:NUM_QUANTIZERS_USED, :, :].cpu()
    return flatten_tokens(semantic_tokens)

def save_waveform(filename, waveform):
  torchaudio.save(filename, waveform[0].detach().cpu(), 16000)

def decode_tokens(tokens):
  unflattened_tokens = unflatten_tokens(tokens)
  return speech_tokenizer.decode(unflattened_tokens)

def save_to_file(tok, filename):
  outputwav = decode_tokens(tok.detach().to(device))
  save_waveform(filename, outputwav)

# Transposing the timestep and code books before flattening to have it be a1, b1, c1 instead of a1, a2, a3, b1, b2, b3,
# since I'm throwing away some of the codebooks, and I also want to be able to generate based on timestep
def flatten_tokens(tokens):
    n_q, B, T = tokens.shape
    transpose_tokens = tokens.transpose(0, 2)
    return transpose_tokens.reshape(B, T * NUM_QUANTIZERS_USED)


def unflatten_tokens(tokens):
    B, L = tokens.shape
    T = L // NUM_QUANTIZERS_USED
    return tokens.reshape(T, B, NUM_QUANTIZERS_USED).transpose(0, 2)



/usr/local/lib/python3.10/dist-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


In [ ]:
from pytube import Playlist

def download_audio_from_playlist(playlist_url, output_path):
    playlist = Playlist(playlist_url)
    for video in playlist.videos:
        audio_stream = video.streams.get_audio_only()
        audio_stream.download(output_path=output_path, filename=video.title + ".mp4")
# Using SolidJJ Stooge for fun
playlist_url = "https://www.youtube.com/watch?v=wvpiSF6nOnU&list=PLLmcRmQRjl9GA8VWVqBCqY2nKZL9dF3RT"
download_audio_from_playlist(playlist_url, 'audio/')

In [ ]:
from moviepy.editor import AudioFileClip
import os
# Length of the clips for training
LENGTH_OF_CLIPS = 10

def convert_mp4_to_wav_clips(mp4_file, output_dir):
  if not os.path.exists(output_dir):
      os.makedirs(output_dir)

  # Convert mp4 to wav and split into 10-second clips
  # Load the audio from the video file
  audio = AudioFileClip(mp4_file)

  # Duration of the audio in seconds
  duration = int(audio.duration)

  # Split the audio into 10-second clips and save
  mp4_output_name = os.path.basename(mp4_file).split('.')[0].replace(' ', '_').replace("#", "num")
  print(mp4_output_name)
  for start in range(0, duration, LENGTH_OF_CLIPS):
      outputpath = os.path.join(output_dir, f'{mp4_output_name}_clip_{start}_{start+LENGTH_OF_CLIPS}.wav')
      if os.path.exists(outputpath):
          continue
      end = min(start + LENGTH_OF_CLIPS, duration)
      clip = audio.subclip(start, end)
      clip.write_audiofile(outputpath, verbose=False, logger=None)

  message = "Conversion and splitting successful."

  print(message)
import os
from tqdm import tqdm
def get_audio_files(directory):
    return [os.path.join(directory, file) for file in os.listdir(directory) if file.endswith('.mp4')]

audio_files = get_audio_files('audio')
print(audio_files)
for af in tqdm(audio_files):
    convert_mp4_to_wav_clips(af, 'MarcBotClips')

["audio/The Three Stooges don't have Uno.mp4", 'audio/The Four Stooges.mp4', 'audio/A STOOGE AMONG US.mp4', "audio/It's A Three Stooges Thanksgiving [ANIMATED].mp4", 'audio/A Three Stooges Thanksgiving.mp4', 'audio/The Three Stooges go Hollywood.mp4', 'audio/The Three Geniuses.mp4', 'audio/The Three Stooges pass the buck.mp4', 'audio/The Three Stooges run for office.mp4', 'audio/The Three Stooges go Job Hunting.mp4', 'audio/The Three Stooges challenge God.mp4', 'audio/The Three Stooges break the 4th wall.mp4', 'audio/The Three Stooges meet “Candice”.mp4', 'audio/The Three Stooges Steal from the Mob.mp4', 'audio/The Three Stooges: The Art of the Trade.mp4', 'audio/Heartbreaking.mp4']


  0%|          | 0/16 [00:00<?, ?it/s]

The_Three_Stooges_don't_have_Uno


  6%|▋         | 1/16 [00:00<00:06,  2.38it/s]

Conversion and splitting successful.
The_Four_Stooges


...

Conversion and splitting successful.
The_Three_Stooges:_The_Art_of_the_Trade


 94%|█████████▍| 15/16 [00:07<00:00,  2.48it/s]

Conversion and splitting successful.
Heartbreaking


100%|██████████| 16/16 [00:07<00:00,  2.03it/s]

Conversion and splitting successful.


In [ ]:
!mkdir testfiles

In [ ]:
from datasets import load_dataset
print("Loading Dataset")
# Select subset from MarcBotClips
audio_dataset = load_dataset("audiofolder", data_dir="MarcBotClips")["train"]

print("Normalizing the waveforms")
audio_dataset = audio_dataset.map(
    lambda x: {
        "original_sampling_rate": x["audio"]["sampling_rate"],
        "audio_array": normalize_waveform(
            torch.tensor(x["audio"]["array"]), x["audio"]["sampling_rate"]
        ),
    },
    remove_columns=["audio"],
    # keep_in_memory=True,
    writer_batch_size=15000,
)

print("Making sure the dataset is in the correct format")
# Making sure that the dataset consists only of 10 second clips at 16000 sample rate.
audio_dataset = audio_dataset.filter(
    lambda x: len(x["audio_array"][0]) ==  speech_tokenizer.sample_rate*LENGTH_OF_CLIPS,
)

print("Tokenizing the waveforms")
audio_dataset = audio_dataset.map(
    lambda x: {"tokens": tokenize(torch.tensor(x["audio_array"]))
               },
    remove_columns=[
        "audio_array",
    ],
    writer_batch_size=15000,
)

# Checking the files to see if the tokenization worked correctly.
for idx, t in enumerate(audio_dataset.select(range(0, 10))):
   save_to_file(torch.tensor(t["tokens"]).to(device), f"testfiles/{idx}_test.wav")

Loading Dataset


Resolving data files:   0%|          | 0/102 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Normalizing the waveforms


Map:   0%|          | 0/102 [00:00<?, ? examples/s]

Making sure the dataset is in the correct format


Filter:   0%|          | 0/102 [00:00<?, ? examples/s]

Tokenizing the waveforms


Map:   0%|          | 0/91 [00:00<?, ? examples/s]

In [ ]:
audio_dataset = audio_dataset.with_format('torch')
audio_dataset = audio_dataset.train_test_split(0.05)
# Setting up the train and test dataloader
train_dataloader = torch.utils.data.DataLoader(audio_dataset['train'], batch_size=batch_size, shuffle=True)
test_dataloader = torch.utils.data.DataLoader(audio_dataset['test'], batch_size=batch_size, shuffle=True)



In [ ]:
from tqdm import tqdm
from mamba_ssm import Mamba
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import torchinfo
import torchaudio
from torch.utils.data import DataLoader
import os
import torch
import torch.nn as nn
from torch.nn import  functional as F
#hyperparams

epochs = 50
lr = 1e-3
block_size = 2000
device = "cuda" if torch.cuda.is_available() else "cpu"
max_iters = 10000
print_iters = 100
eval_iters = 10
eval_interval = 300
n_embed=384
n_heads = 6
n_layers = 6
dropout = 0.2
vocab_size=1024
from tqdm import tqdm

# ---------




class SelfAttentionHead(nn.Module):
  def __init__(self, head_size):
    super().__init__()
    self.keys = nn.Linear(n_embed, head_size)
    self.queries = nn.Linear(n_embed, head_size)
    self.values = nn.Linear(n_embed, head_size)
    self.head_size = head_size
    self.n_embed = n_embed
    self.register_buffer('tril', torch.tril(torch.ones((block_size,block_size))).to(device))
    self.dropout = nn.Dropout(dropout)

  def forward(self, x):
    B,T,C = x.shape
    k = self.keys(x) # (B,T,C_h)
    q = self.queries(x) # (B,T,C_h)
    v = self.values(x) # (B,T,C_h)
    wei = k @ q.transpose(-1,-2) * C**(-0.5)# (B,T,T)
    wei = wei.masked_fill( self.tril[:T,:T]==0, float('-inf'))
    # wei = F.softmax(wei, dim=-1) # (B,T,T)
    wei = torch.log(torch.exp(wei)+1) # (B,T,T)
    wei = self.dropout(wei)
    out = wei @ v # (B,T,C_h)
    return out


class MultiHeadAttention(nn.Module):
  def __init__(self, n_heads, head_size) -> None:
    super().__init__()
    self.heads = nn.ModuleList([SelfAttentionHead(head_size) for _ in range(n_heads)])
    self.proj = nn.Linear(n_embed, n_embed)
    self.dropout = nn.Dropout(dropout)

  def forward(self, x):
    B,T,C = x.shape
    out = torch.cat([head(x) for head in self.heads], dim=-1)
    out = self.proj(out)
    out = self.dropout(out)
    return out

class FeedForward(nn.Module):
  def __init__(self, n_embed) -> None:
    super().__init__()
    self.ffn = nn.Sequential(
      nn.Linear(n_embed, 4*n_embed),
      nn.ReLU(),
      nn.Linear(4*n_embed, n_embed),
      nn.Dropout(dropout),
    )
  def forward(self, x):
    return self.ffn(x)

class Block(nn.Module):
  def __init__(self, n_embed, n_heads) -> None:
    super().__init__( )
    self.sa_head = Mamba(
      # This module uses roughly 3 * expand * d_model^2 parameters
      d_model=n_embed, # Model dimension d_model
      d_state=16,  # SSM state expansion factor
      d_conv=4,    # Local convolution width
      expand=1,    # Block expansion factor
  ).to("cuda")
    self.ffn = FeedForward(n_embed)
    self.ln1 = nn.LayerNorm(n_embed)
    self.ln2 = nn.LayerNorm(n_embed)


  def forward(self, x):
    x = x + self.sa_head(self.ln1(x))
    x = x + self.ffn(self.ln2(x))

    return x

class MambaAudioModel(nn.Module):
  def __init__(self,vocab_size):
    super().__init__()
    self.token_embedding_table = nn.Embedding(vocab_size,n_embed)
    self.position_embedding_table = nn.Embedding(block_size,n_embed)
    self.lm_head = nn.Linear(n_embed,vocab_size)
    self.ffn = FeedForward(n_embed)
    print("layers", n_layers)
    self.blocks = nn.Sequential(*[Block(n_embed,n_heads=n_heads) for _ in range(n_layers)])


  def forward(self, idx, targets=None):
    # idx = idx[:,-block_size:]
    B,T = idx.shape
    tok_emb = self.token_embedding_table(idx) # (B,T, C_e)
    pos_emb = self.position_embedding_table(torch.arange(T,device=device)) # (T, C_e)
    x = tok_emb + pos_emb # (B,T,Q, C_e)
    x = self.blocks(x) # (B,T,Q, C_e)
    logits = self.lm_head(x) # (B,T,vocab_size)
    if targets is None:
      loss = None
    else:
      B,T,C = logits.shape
      logits = logits.view(B*T,C)
      targets = targets.view(B*T)
      loss = F.cross_entropy(logits, targets)
      logits = logits.view(B,T,C)
    return logits, loss


def estimate_test_loss(model, dataset):
  model.eval()
  test_losses = []
  with torch.no_grad():
    for tokens in tqdm(dataset['test']['tokens']):
      tokens = tokens.to(device)
      x = tokens[:,:-1].contiguous()
      y = tokens[:,1:].contiguous()
      logits, loss = model(x,y )
      test_losses.append(loss)
  model.train()
  return sum(test_losses)/len(test_losses)




def produce_wav(filename, model, example):
  firstbit = example.shape[-1]//2
  tokens = example[:firstbit]
  tokens = tokens.reshape(1,firstbit)
  max_new_tokens=example.shape[-1]-firstbit
  idx = tokens.to(device)
  idx_next = []
  for i in tqdm(range(max_new_tokens)):
    idx_cond = idx[:,-block_size:]
    logits, loss = model(idx_cond)
    last_timestep = logits[:,-1,:]
    probs = F.softmax(last_timestep, dim=1)
    next_index = torch.multinomial(probs, num_samples=1)
    idx = torch.cat((idx, next_index), dim=1)

  print(idx.shape,idx)
  # assert(torch.allclose(idx[0][:firstbit], example[:firstbit]))
  save_to_file(idx, f"{filename}_test.wav")
  save_to_file(example.unsqueeze(0), f"{filename}_input.wav")
  wandb.save(f"{filename}_test.wav")
  wandb.save( f"{filename}_input.wav")

def save_model_checkpoint(ind, model, dataset):
  model_path = f"checkpoints/{ind}_model.pt"
  torch.save(model.state_dict(), model_path)
  wandb.save(model_path)
  train_filename = f"checkpoints/{ind}_train"
  test_filename = f"checkpoints/{ind}_test"
  produce_wav(train_filename, model, dataset['train'][0]['tokens'][0])
  produce_wav(test_filename, model, dataset['test'][0]['tokens'][0])


In [ ]:
import wandb
import time
import torchinfo
from tqdm.notebook import tqdm
wandb.init(project="speech_tokenizer_test")
model = MambaAudioModel(vocab_size).to(device)
optimizer = torch.optim.AdamW(model.parameters(),lr=lr)
print(torchinfo.summary(model))
model.train()
ind = 0
start_time = time.time()

for epoch in tqdm(range(epochs)):
  for batch in tqdm(train_dataloader):
    if batch['tokens'].shape[-1] != block_size:
      continue
    batch_size = batch['tokens'].shape[0]
    tokens = batch['tokens'].to(device).reshape(batch_size,block_size)

    x = tokens[:,:-1].contiguous()
    y = tokens[:,1:].contiguous()
    logits, loss = model(x,y)
    if ind % 10 == 0:
      print(loss)
      wandb.log({"Training Loss":loss.item(), "Iteration":ind, "Epoch":epoch, "time_elapsed":time.time()-start_time})
    if ind % 500 == 0:
      tl = estimate_test_loss(model, audio_dataset)
      print("testloss", tl)
      wandb.log({"Testing Loss":tl.item(), "Iteration":ind, "Epoch":epoch})
    if ind % 1000 == 0:
      save_model_checkpoint(ind, model, audio_dataset)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    # Prevents gradient explosion.
    torch.nn.utils.clip_grad.clip_grad_norm_(model.parameters(), 1.0)
    optimizer.step()
    ind += 1

model_path="overfittedmodel.pt"
torch.save(model.state_dict(), model_path)
wandb.save(model_path)
wandb.finish()
print(torchinfo.summary(model))



wandb: Currently logged in as: lukasnel2084. Use `wandb login --relogin` to force relogin


layers 6
Layer (type:depth-idx)                   Param #
MambaAudioModel                          --
├─Embedding: 1-1                         393,216
├─Embedding: 1-2                         768,000
├─Linear: 1-3                            394,240
├─FeedForward: 1-4                       --
│    └─Sequential: 2-1                   --
│    │    └─Linear: 3-1                  591,360
│    │    └─ReLU: 3-2                    --
│    │    └─Linear: 3-3                  590,208
│    │    └─Dropout: 3-4                 --
├─Sequential: 1-5                        --
│    └─Block: 2-2                        --
│    │    └─Mamba: 3-5                   481,920
│    │    └─FeedForward: 3-6             1,181,568
│    │    └─LayerNorm: 3-7               768
│    │    └─LayerNorm: 3-8               768
│    └─Block: 2-3                        --
│    │    └─Mamba: 3-9                   481,920
│    │    └─FeedForward: 3-10            1,181,568
│    │    └─LayerNorm: 3-11              768
│    │    

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/22 [00:00<?, ?it/s]

tensor(7.3519, device='cuda:0', grad_fn=<NllLossBackward0>)


...

tensor(0.0009, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.0015, device='cuda:0', grad_fn=<NllLossBackward0>)


  0%|          | 0/22 [00:00<?, ?it/s]

tensor(0.0016, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.0027, device='cuda:0', grad_fn=<NllLossBackward0>)


Epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
Iteration,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
Testing Loss,▁▆█
Training Loss,█▇▅▅▄▄▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
time_elapsed,▁▂▂▂▂▂▃▃▃▃▃▃▃▄▄▄▄▄▄▄▅▅▅▅▅▅▅▆▆▆▆▆▆▆▇▇▇███
Epoch,49
Iteration,1090
Testing Loss,14.99256
Training Loss,0.00266
time_elapsed,249.16589


Layer (type:depth-idx)                   Param #
MambaAudioModel                          --
├─Embedding: 1-1                         393,216
├─Embedding: 1-2                         768,000
├─Linear: 1-3                            394,240
├─FeedForward: 1-4                       --
│    └─Sequential: 2-1                   --
│    │    └─Linear: 3-1                  591,360
│    │    └─ReLU: 3-2                    --
│    │    └─Linear: 3-3                  590,208
│    │    └─Dropout: 3-4                 --
├─Sequential: 1-5                        --
│    └─Block: 2-2                        --
│    │    └─Mamba: 3-5                   481,920
│    │    └─FeedForward: 3-6             1,181,568
│    │    └─LayerNorm: 3-7               768
│    │    └─LayerNorm: 3-8               768
│    └─Block: 2-3                        --
│    │    └─Mamba: 3-9                   481,920
│    │    └─FeedForward: 3-10            1,181,568
│    │    └─LayerNorm: 3-11              768
│    │    └─LayerNo

In [ ]:
model_path="overfittedmodel.pt"
torch.save(model.state_dict(), model_path)
wandb.save(model_path)
wandb.finish()

Error: You must call wandb.init() before wandb.save()

In [ ]:
def unconditional_generation(model):
    idx = torch.tensor([[10,]]).to(device)
    max_new_tokens = 1999
    idx_next = []
    for i in tqdm(range(max_new_tokens)):
        idx_cond = idx[:,-block_size:]
        logits, loss = model(idx_cond)
        last_timestep = logits[:,-1,:]
        probs = F.softmax(last_timestep, dim=1)
        next_index = torch.multinomial(probs, num_samples=1)
        idx = torch.cat((idx, next_index), dim=1)

    save_to_file(idx, f"unconditional_test.wav")

In [ ]:
for idx, t in enumerate(audio_dataset['test'].select(range(0, 10))):
   save_to_file(torch.tensor(t["tokens"]).to(device), f"testfiles/{idx}_test.wav")

In [ ]:
!mkdir testfiles